<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 2.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=7cd2deee85f7872c57a1166b390f29a31b926a88675873276df0cf4d31739027
  Stored in directory: /Users/lucabalde/Library/Caches/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [3]:
import folium
import wget
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [5]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [............................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Recenter/zoom to see all sites nicely (optional)
site_map = folium.Map(location=[launch_sites_df['Lat'].mean(),
                                launch_sites_df['Long'].mean()],
                      zoom_start=5)

# Add a circle + text label marker for each launch site
for _, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    
    # Circle with popup
    folium.Circle(
        location=coord,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(
        folium.Popup(row['Launch Site'])
    ).add_to(site_map)

    # Text label at the same coordinate
    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(120, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px;color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Show the map in Jupyter
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


### Findings:

Proximity to the Equator

Not all launch sites are extremely close to the Equator.

While some SpaceX sites (e.g., Cape Canaveral, Kennedy Space Center) are at relatively low latitudes (around 28°N), others—like Vandenberg in California (~34°N)—are farther from the Equator.

Sites closer to the Equator benefit from the Earth’s rotational speed, which gives rockets an extra velocity boost for equatorial orbits, but higher-latitude sites are chosen for polar or sun-synchronous orbits.

Proximity to the Coast

All SpaceX launch sites are indeed very close to the coast.

Coastal proximity allows rockets to launch over the ocean, reducing risks to populated areas from debris or failed launches, and provides a safe downrange for stages or boosters returning to land or drone ships.

Explanation:
SpaceX chooses coastal sites for safety and logistical reasons, and lower-latitude sites when missions benefit from extra rotational velocity. However, missions requiring different orbital inclinations (like polar orbits) are launched from higher-latitude coastal sites. The map visualization highlights this pattern clearly—every site is next to a large body of water, but only some are near the Equator.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
from folium.plugins import MarkerCluster

# Ensure expected columns exist; cast class to int for safety
spacex_df['class'] = spacex_df['class'].astype(int)

# Create (or reuse) the base map; if reusing, skip this line
# site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=5)

# Create a MarkerCluster and add to map
outcomes_cluster = MarkerCluster(name="Launch Outcomes").add_to(site_map)

# Helper for color by class
def outcome_color(c):
    return "green" if c == 1 else "red"

# Add one marker per launch record
for _, r in spacex_df.iterrows():
    folium.CircleMarker(
        location=[r['Lat'], r['Long']],
        radius=5,
        color=outcome_color(r['class']),
        fill=True,
        fill_opacity=0.8,
        tooltip=f"{r['Launch Site']} | {'Success' if r['class']==1 else 'Failure'}"
    ).add_to(outcomes_cluster)

# Optional: site labels again (if not already on the map)
for _, s in spacex_df.groupby('Launch Site', as_index=False).first().iterrows():
    folium.map.Marker(
        [s['Lat'], s['Long']],
        icon=DivIcon(
            icon_size=(120, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px;color:#333;background:#fff;padding:2px 4px;border:1px solid #ccc;border-radius:3px;"><b>{s["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Add layer control to toggle clusters/tiles
folium.LayerControl().add_to(site_map)

# Simple legend
legend_html = """
<div style="
    position: fixed; bottom: 30px; left: 30px; z-index: 9999;
    background: white; padding: 8px 12px; border: 1px solid #ccc; border-radius: 6px;
    font-size: 12px; box-shadow: 0 2px 6px rgba(0,0,0,0.2);">
  <b>Outcome Legend</b><br>
  <span style="display:inline-block;width:10px;height:10px;background:green;border-radius:50%;margin-right:6px;"></span>Success<br>
  <span style="display:inline-block;width:10px;height:10px;background:red;border-radius:50%;margin-right:6px;"></span>Failure
</div>
"""
site_map.get_root().html.add_child(folium.Element(legend_html))

# Display
site_map


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Define the function
def set_marker_color(cls):
    if cls == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create the new column
spacex_df['marker_color'] = spacex_df['class'].apply(set_marker_color)

# Preview the dataframe
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [14]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [17]:
from folium.plugins import MarkerCluster

# Create the cluster and attach to the map
marker_cluster = MarkerCluster(name="Launch Outcomes").add_to(site_map)

# (Optional) ensure the color column exists
if 'marker_color' not in spacex_df.columns:
    spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'})

# Add one marker per launch
for _, row in spacex_df.iterrows():
    outcome = 'Success' if int(row['class']) == 1 else 'Failure'
    folium.Marker(
        location=[row['Lat'], row['Long']],
        tooltip=f"{row['Launch Site']} | {outcome}",
        popup=f"{row['Launch Site']} – {outcome}",
        icon=folium.Icon(
            color='white',                 # marker body
            icon_color=row['marker_color'],# rocket glyph color (green/red)
            icon='rocket',
            prefix='fa'
        )
    ).add_to(marker_cluster)

# Optional: toggle layers
folium.LayerControl().add_to(site_map)

site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
from math import sin, cos, sqrt, atan2, radians

# Haversine distance (km)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# 1) Pick a launch site
print("Available sites:", launch_sites_df['Launch Site'].unique())
site_name = launch_sites_df['Launch Site'].iloc[0]   # or set explicitly, e.g. "KSC LC-39A"

site_row = launch_sites_df[launch_sites_df['Launch Site'] == site_name].iloc[0]
launch_site_lat, launch_site_lon = float(site_row['Lat']), float(site_row['Long'])

# 2) Paste the coastline coordinates you read from the map
coastline_lat = 28.56367   # <-- replace with your measured Lat
coastline_lon = -80.57163  # <-- replace with your measured Lon

# 3) Compute distance
distance_km = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance {site_name} → Coastline: {distance_km:.2f} km")

# 4) Visualize: add coastline point, line, and distance label
folium.Marker(
    [coastline_lat, coastline_lon],
    tooltip=f"Coastline Point\n({coastline_lat:.5f}, {coastline_lon:.5f})",
    icon=folium.Icon(color='blue', icon='anchor', prefix='fa')
).add_to(site_map)

# Line between site and coastline
folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)],
    weight=3,
    tooltip=f"{site_name} → Coastline: {distance_km:.2f} km"
).add_to(site_map)

# Midpoint distance label
mid_lat = (launch_site_lat + coastline_lat) / 2
mid_lon = (launch_site_lon + coastline_lon) / 2
folium.map.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(180, 24),
        icon_anchor=(0, 0),
        html=f'<div style="background:white;padding:2px 6px;border:1px solid #ccc;border-radius:4px;font-size:12px;">{site_name} → Coastline: {distance_km:.2f} km</div>'
    )
).add_to(site_map)

site_map


Available sites: ['CCAFS LC-40' 'CCAFS SLC-40' 'KSC LC-39A' 'VAFB SLC-4E']
Distance CCAFS LC-40 → Coastline: 0.58 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [22]:
# Compute distance (km)
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a label marker at the coastline point
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(140, 24),
        icon_anchor=(0, 0),
        html=f'<div style="font-size:12px;color:#d35400;background:white;'
             f'padding:2px 6px;border:1px solid #ccc;border-radius:4px;">'
             f'<b>{distance_coastline:.2f} KM</b></div>'
    ),
    tooltip=f"Coastline ({coastline_lat:.5f}, {coastline_lon:.5f})"
)

site_map.add_child(distance_marker)
site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [23]:
# Coordinates: (lat, lon) pairs
coordinates = [
    (launch_site_lat, launch_site_lon),
    (coastline_lat, coastline_lon)
]

# Create the polyline (increase weight if you want it thicker)
lines = folium.PolyLine(
    locations=coordinates,
    weight=3,
    tooltip=f"Site → Coastline: {distance_coastline:.2f} km"  # optional
)

# Add to map and display
site_map.add_child(lines)
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [24]:
# ---- 0) Make sure you already have these defined ----
# launch_site_lat, launch_site_lon  (from your chosen launch site)
# site_map                          (your Folium map)
# calculate_distance(lat1, lon1, lat2, lon2)  (from earlier)

import folium
from folium.features import DivIcon

# ---- 1) Insert the POI coordinates you captured with MousePosition ----
# Replace the placeholder values with your measured coordinates
poi_coords = {
    "Closest City":    {"lat": 28.53834, "lon": -81.37924, "icon": "building"},  # example: Orlando
    "Closest Railway": {"lat": 28.52000, "lon": -80.70000, "icon": "train"},
    "Closest Highway": {"lat": 28.54500, "lon": -80.65000, "icon": "road"},
    # Add more if needed:
    # "Closest Hospital": {"lat": ..., "lon": ..., "icon": "plus"}
}

# ---- 2) Small helper to add marker + line + distance label ----
def add_proximity(label, lat, lon, icon_name="map-marker"):
    dist_km = calculate_distance(launch_site_lat, launch_site_lon, lat, lon)

    # POI marker
    folium.Marker(
        [lat, lon],
        tooltip=f"{label}\n({lat:.5f}, {lon:.5f})",
        popup=f"{label} – {dist_km:.2f} km",
        icon=folium.Icon(color="blue", icon=icon_name, prefix="fa")
    ).add_to(site_map)

    # Distance label at the POI
    folium.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(150, 24),
            icon_anchor=(0, 0),
            html=(f'<div style="font-size:12px;color:#d35400;background:white;'
                  f'padding:2px 6px;border:1px solid #ccc;border-radius:4px;">'
                  f'<b>{dist_km:.2f} KM</b></div>')
        )
    ).add_to(site_map)

    # Line from site to POI
    folium.PolyLine(
        locations=[(launch_site_lat, launch_site_lon), (lat, lon)],
        weight=3,
        tooltip=f"Launch site → {label}: {dist_km:.2f} km"
    ).add_to(site_map)

    return dist_km

# ---- 3) Add all proximities and print a quick summary ----
dist_summary = {}
for label, meta in poi_coords.items():
    d = add_proximity(label, meta["lat"], meta["lon"], meta.get("icon", "map-marker"))
    dist_summary[label] = d

# Optional: see the distances in the notebook output
for k, v in dist_summary.items():
    print(f"{k}: {v:.2f} km")

# Display the map
site_map


Closest City: 78.39 km
Closest Railway: 12.87 km
Closest Highway: 7.35 km


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


### Findings: 

Railways (~12.87 km): Yes, reasonably close. That’s near enough to support logistics (heavy parts, propellants) even if the spur isn’t right at the pad.

Highways (~7.35 km): Yes, close. Road access is essential for daily ops, workforce, emergency services, and moving oversized hardware.

Coastline (likely a few km in your map): Yes, very close. Pads are intentionally on the coast so flight paths go over water, creating safe downrange debris/recovery zones.

Cities (~78.39 km): Yes, they keep distance. Large standoff reduces risk to populations (debris, acoustic loads, blast radius) and eases range safety constraints.

Why this pattern appears

Safety: Coastal siting + distance from cities minimize risk during nominal flight and anomalies.

Operations: Highway/rail proximity simplifies transport of large, heavy components and ongoing supply.

Mission profiles: Coastal ranges also provide clear downrange corridors and flexibility for varied orbital inclinations.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
